In [11]:
pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install webdriver-manager


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install selenium webdriver_manager


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [15]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [16]:
chrome_driver_path='/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/chromedriver_py/chromedriver_mac-x64'

In [17]:
service = Service(ChromeDriverManager().install())

In [18]:
driver = webdriver.Chrome(service=service)

In [19]:
driver.get("https://attack.mitre.org")

In [20]:
pip install beautifulsoup4


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
from bs4 import BeautifulSoup

In [22]:
import csv

In [23]:

# Wait until the <div> with ID 'tour-matrix-container' is present
div_container = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'tour-matrix-container'))
)

# Extract the HTML content of the <div>
div_html = div_container.get_attribute('innerHTML')

# Parse the HTML content of the <div> with BeautifulSoup
soup = BeautifulSoup(div_html, 'html.parser')

# Find the table with class 'matrix side' inside the <div>
table = soup.find('table', class_='matrix side')

if table:
    # Open a CSV file to write the data
    with open('techniques.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['TECHNIQUE ID', 'subtechnique id', 'tactic name', 'technique name', 'sub technique name', 'platform', 'Description'])
        
        # Assuming the table rows are in <tr> and cells in <td>
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 7:
                # Extract details from cells
                technique_id = cells[0].text.strip()
                subtechnique_id = cells[1].text.strip()
                tactic_name = cells[2].text.strip()
                technique_name = cells[3].text.strip()
                sub_technique_name = cells[4].text.strip()
                platform = cells[5].text.strip()
                description = cells[6].text.strip()
                
                # Write the extracted details to the CSV file
                writer.writerow([technique_id, subtechnique_id, tactic_name, technique_name, sub_technique_name, platform, description])
            else:
                print(f"Skipping row with insufficient cells: {len(cells)} cells found.")
else:
    print("Table not found within the <div>.")


Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 4 cells found.
Skipping row with insufficient cells: 1 cells found.
Skipping row with insufficient cells: 2 cells 

In [24]:
pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install torch


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
import torch
print(torch.__version__)

2.3.1


In [31]:
from transformers import pipeline
import pandas as pd

# Load a pre-trained model and tokenizer
generator = pipeline('text-generation', model='gpt2')

def generate_descriptions(technique_id, aggregated_description):
    descriptions = []
    prompt = f"Generate different descriptions for the following technique ID: {technique_id}. Description: {aggregated_description}"
    
    while len(descriptions) < 50:
        response = generator(prompt, max_length=150, num_return_sequences=1)
        descriptions.append(response[0]['generated_text'].strip())
    
    return descriptions

# Load the CSV
df = pd.read_csv('MitreScrappedData.csv')

# Group by Technique ID and Aggregate Descriptions
grouped = df.groupby('Technique')['Description'].apply(lambda x: ' '.join(x)).reset_index()

# Generate Descriptions
generated_data = []

for index, row in grouped.iterrows():
    technique_id = row['Technique']
    aggregated_description = row['Description']
    descriptions = generate_descriptions(technique_id, aggregated_description)
    
    for desc in descriptions:
        generated_data.append({
            'Technique': technique_id,
            'Description': desc
        })

# Save to CSV
generated_df = pd.DataFrame(generated_data)
generated_df.to_csv('generated_descriptions.csv', index=False)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.